<a href="https://colab.research.google.com/github/m25c1049/generative_AI/blob/main/kadai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install langchain-core
!pip install langchain-community
!pip install langchain-huggingface
!pip install llama-cpp-python
!pip install huggingface-hub
!pip install sentence-transformers
!pip install chromadb
!pip install beautifulsoup4
!pip install lxml
!pip install requests
!pip install numpy
!pip install transformers
!pip install torch
!pip install tokenizers
# google-colab の依存関係を解決するため、requests==2.32.4 を明示的にインストールします。
# これにより、langchain-community に関連する警告が表示される可能性があります。
!pip install requests==2.32.4 --force-reinstall
!pip install newsapi-python

  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)
Using cached charset_normalizer-3.4.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl (151 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Succe

News API を使用するにはAPIキーが必要です。 [News API のウェブサイト](https://newsapi.org/) でAPIキーを取得できます。

取得したAPIキーは、Colab の左パネルにある「🔑」アイコンのシークレットマネージャーに `NEWS_API_KEY` という名前で保存してください。

# Task
News API を使用して最新のニュースを取得し、その結果をもとにユーザーの質問に回答できるように、LangChain と Llama モデルを用いた既存のノートブックを更新してください。

## News api からニュースを取得

### Subtask:
ユーザーの質問に関連するキーワードで News API を呼び出し、ニュース記事のリストを取得します。


**Reasoning**:
Modify the existing code to use the user's question as a keyword for fetching news articles from the News API.



**Reasoning**:
The error indicates that `api_key` and `newsapi` are not defined. These variables were defined in a previous cell but are not accessible in the current one. I need to include the code that initializes the NewsApiClient and retrieves the API key.



**Reasoning**:
The error indicates that the `newsapi` module is not found. Although a previous cell installed the required packages, the current environment might not have it loaded. I need to explicitly install the `newsapi-python` package again to ensure it's available in the current environment.



In [2]:
!pip install newsapi-python

**Reasoning**:
Now that the `newsapi-python` package is installed, I can proceed with the code to initialize the NewsApiClient, get the API key, fetch news articles based on the user's question, and store the results.



**Reasoning**:
The error "ValueError: invalid language" indicates that the language code 'jp' is not a valid option for the `get_everything` endpoint in the News API. I need to change the language parameter to 'ja' which is the correct ISO 639-1 code for Japanese.



**Reasoning**:
The previous command failed because the `langchain_community` library was not found. Although it was installed earlier, the kernel might have reset or the environment changed. Re-installing the necessary libraries within the current context should resolve this.



In [3]:
!pip install langchain-community
!pip install langchain-huggingface
!pip install chromadb

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.3 which is incompatible.


**Reasoning**:
Now that the necessary libraries are installed, attempt to create the vector store again using the chunked news documents and the embedding model.



## Retriever

### Subtask:
更新または新規作成したベクトルDB を使用する LangChain の Retriever を設定します。


**Reasoning**:
Set up the LangChain Retriever using the existing vector_store.



In [4]:
# Check if vector_store is defined and not None
if 'vector_store' in locals() and vector_store is not None:
    # Set up the vector DB as a LangChain Retriever
    # Configure the number of chunks to extract using search_kwargs
    retriever = vector_store.as_retriever(search_kwargs={"k": 2})

    print("Retriever set up successfully.")
else:
    print("Vector store is not available. Cannot set up retriever.")

Vector store is not available. Cannot set up retriever.


## News api からニュースを取得

### Subtask:
ユーザーの質問に関連するキーワードで News API を呼び出し、ニュース記事のリストを取得します。


**Reasoning**:
Import the necessary libraries and retrieve the API key, then make the call to the News API with the user's question as the keyword. Check for the presence of articles and print relevant information, storing the articles in a list.



In [6]:
from huggingface_hub import hf_hub_download
from langchain_community.llms import LlamaCpp
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Define LLM parameters (referencing values from the original notebook)
CONTEXT_SIZE = 2048
LLM_REPO_ID = "mmnga/ELYZA-japanese-Llama-2-7b-instruct-gguf"
LLM_FILE = "ELYZA-japanese-Llama-2-7b-instruct-q4_K_S.gguf"

# LLMを生成
# Check if model_path is defined, otherwise download it
if 'model_path' not in locals():
    model_path = hf_hub_download(repo_id=LLM_REPO_ID, filename=LLM_FILE)

llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=128,
    n_ctx=CONTEXT_SIZE,
    f16_kv=True,
    verbose=True,
    seed=0
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--mmnga--ELYZA-japanese-Llama-2-7b-instruct-gguf/snapshots/2d708f9c52bde588049a494e95b986f5bedba76f/ELYZA-japanese-Llama-2-7b-instruct-q4_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = ELYZA-japanese-Llama-2-7b-instruct
llama_model_loader: - kv   2:       general.source.hugginface.repository str              = elyza/ELYZA-japanese-Llama-2-7b-

In [7]:
from newsapi import NewsApiClient
from google.colab import userdata
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# シークレットマネージャーからAPIキーを取得
api_key = userdata.get('NEWS_API_KEY')

if api_key:
    newsapi = NewsApiClient(api_key=api_key)

    # ユーザーが質問を入力
    question = input("質問を入力してください: ")

    # LLMが定義されているか確認 (キーワード抽出は維持)
    if 'llm' in locals() and llm is not None:
        print("\nAIで質問からキーワードを抽出中...")
        # キーワード抽出のためのプロンプトテンプレート
        keyword_template = """<s>[INST] <<SYS>>
以下の質問内容から、News API でニュースを検索するのに最適なキーワードを複数セット（各セット最大3つまで、カンマ区切り）抽出してください。複数のキーワードセットは、改行で区切って回答してください。
<</SYS>>

質問内容：{question} [/INST]"""

        keyword_prompt = ChatPromptTemplate.from_template(keyword_template)
        output_parser = StrOutputParser()
        keyword_chain = keyword_prompt | llm | output_parser

        try:
            # LLMにキーワード抽出を依頼
            keyword_response = keyword_chain.invoke({"question": question})
            extracted_keyword_sets_str = keyword_response.strip()
            print(f"抽出されたキーワードセット:\n{extracted_keyword_sets_str}")

            # 抽出されたキーワードセットをリストに変換
            extracted_keyword_sets = [s.strip() for s in extracted_keyword_sets_str.split('\n') if s.strip()]
            if not extracted_keyword_sets:
                 extracted_keyword_sets = [question] # キーワードセットが抽出されなかった場合は元の質問を使用

        except Exception as e:
            print(f"キーワード抽出中にエラーが発生しました: {e}")
            extracted_keyword_sets = [question] # エラーの場合は元の質問を使用

    else:
        print("LLM (llm変数) が定義されていないため、キーワード抽出はスキップされました。")
        extracted_keyword_sets = [question] # LLMがない場合は元の質問を使用


    all_articles_from_all_queries = []
    successful_query = None

    # 抽出されたキーワードセット（または元の質問）でNews APIを呼び出し、ニュース記事を取得
    for query in extracted_keyword_sets:
        print(f"\nNews APIで検索中: '{query}'")
        try:
            all_articles = newsapi.get_everything(q=query,
                                                sort_by='relevancy',
                                                page_size=10) # 取得する記事数を10件に制限

            if all_articles and 'articles' in all_articles and all_articles['articles']:
                print(f"'{query}'に関するニュースが見つかりました ({len(all_articles['articles'])}件)。")
                all_articles_from_all_queries.extend(all_articles['articles'])
                successful_query = query # 少なくとも1つのクエリで記事が見つかったことを記録
                # 最初の成功したクエリで見つかった記事を使用し、他のクエリでの検索はスキップすることも検討可能
                # 例: if successful_query: break

            else:
                print(f"'{query}'に関するニュースは見つかりませんでした。")

        except Exception as e:
            print(f"News API 呼び出し中にエラーが発生しました: {e}") # 日本語に修正

    # 取得したニュース記事のリストを表示（確認のため）
    # フィルタリング機能を削除し、取得した全ての記事（重複排除のみ行う）を使用する
    filtered_articles = []
    if all_articles_from_all_queries:
        print(f"\n取得したニュース (重複排除後 - {len(all_articles_from_all_queries)}件):")
        # 重複排除のため、URLをキーとしてセットを使用
        processed_urls = set()
        unique_articles = []
        for article in all_articles_from_all_queries:
             article_url = article.get('url', '')
             if article_url and article_url not in processed_urls:
                 unique_articles.append(article)
                 processed_urls.add(article_url)
        filtered_articles = unique_articles

        if filtered_articles:
             for article in filtered_articles:
                print(f"- {article.get('title', 'タイトルなし')} ({article.get('source', {}).get('name', 'ソース不明')})")
        else:
            print("ニュースが見つかりませんでした。")


    else:
        print(f"'{question}'に関するニュースは見つかりませんでした。")
        # ニュースが見つからなかった場合は空のリストを設定
        filtered_articles = []


    # 取得した（フィルタリング後の）ニュース記事をLangChainで処理できるように、適切な形式に変換して変数に格納する必要がある。
    # この部分は次のサブタスクで実装するため、ここでは一旦articlesのリストを変数に格納するのみとする。
    news_articles_list = filtered_articles

else:
    print("APIキーが設定されていません。シークレットマネージャーに NEWS_API_KEY を設定してください。")
    # APIキーがない場合は空のリストを設定
    news_articles_list = []

# このサブタスクではNews APIからニュース記事を取得するところまでなので、
# LLMチェインの実行部分はコメントアウトするか、次のサブタスクに引き継ぐ。
# for s in chain.stream(question):
#     print(s, end="", flush=True)

質問を入力してください: ロシア・ウクライナの最新情勢を教えて

AIで質問からキーワードを抽出中...


llama_perf_context_print:        load time =   59557.12 ms
llama_perf_context_print: prompt eval time =   59555.62 ms /   168 tokens (  354.50 ms per token,     2.82 tokens per second)
llama_perf_context_print:        eval time =   41620.25 ms /    63 runs   (  660.64 ms per token,     1.51 tokens per second)
llama_perf_context_print:       total time =  101251.29 ms /   231 tokens
llama_perf_context_print:    graphs reused =         75


抽出されたキーワードセット:
* ロシア
* ウクライナ
* 戦争
* 平和
* 欧州安全保障協力機構
* プーチン
* ポロシェンコ

News APIで検索中: '* ロシア'
'* ロシア'に関するニュースが見つかりました (10件)。

News APIで検索中: '* ウクライナ'
'* ウクライナ'に関するニュースが見つかりました (10件)。

News APIで検索中: '* 戦争'
'* 戦争'に関するニュースが見つかりました (10件)。

News APIで検索中: '* 平和'
'* 平和'に関するニュースが見つかりました (10件)。

News APIで検索中: '* 欧州安全保障協力機構'
'* 欧州安全保障協力機構'に関するニュースが見つかりました (10件)。

News APIで検索中: '* プーチン'
'* プーチン'に関するニュースが見つかりました (10件)。

News APIで検索中: '* ポロシェンコ'
'* ポロシェンコ'に関するニュースは見つかりませんでした。

取得したニュース (重複排除後 - 60件):
- トランプ大統領「ウクライナは全領土を取り戻す立場にあると思う」「ロシアは張り子の虎だ」 (Itainews.com)
- 米国、中印への高関税をG7財務相会合で要請　ロシアへの圧力強化 (Asahi.com)
- ロシア無人機、ルーマニアにも領空侵犯か　ウクライナ「偶然でない」 (Asahi.com)
- ロシアから訪日客が急増　関係悪化でも観光、買い物、ビジネスで (Asahi.com)
- ロシア産原油の上限価格引き下げへ　ウクライナ侵略で政府が制裁強化 (Asahi.com)
- NATO「ロシアに全責任」と声明　加盟国で相次ぐ領空侵犯を非難 (Asahi.com)
- 「中国、インドに100%関税を」　トランプ氏米欧に要求、英紙報道 (Asahi.com)
- 米国「ロシアの意志に疑念」　ウクライナ攻撃受けた安保理会合で発言 (Asahi.com)
- プーチン氏、外国軍のウクライナ駐留を拒否　「安全の保証」巡り反発 (Asahi.com)
- NATO加盟国は「ロシア産原油禁輸を」　トランプ氏が欧州に要求 (Asahi.com)
- トランプ氏がゼレンスキー氏と協議へ　中ロ首脳とも「関係良好

## ニュース記事をドキュメント形式に変換

### Subtask:
取得したニュース記事のタイトルと内容を LangChain の `Document` オブジェクトのリストに変換します。


**Reasoning**:
Convert the list of news article dictionaries into a list of LangChain Document objects, including relevant metadata, and display the first few to verify.



In [8]:
from langchain_core.documents import Document

# Create an empty list to store the converted Document objects
news_documents = []

# Check if news_articles_list is defined and not empty before proceeding
if 'news_articles_list' in locals() and news_articles_list:
    # Loop through each article (dictionary) in the list
    for article in news_articles_list:
        # Create a Document object for each article
        # Combine title and description for page_content
        # Include source name, published date, and URL in metadata
        document = Document(
            page_content=(article.get('title', '') or '') + "\n" + (article.get('description', '') or ''),
            metadata={
                'source': article.get('source', {}).get('name', 'Unknown Source'),
                'publishedAt': article.get('publishedAt', 'Unknown Date'),
                'url': article.get('url', 'No URL provided')
            }
        )
        # Add the created Document object to the list
        news_documents.append(document)

# Display the first few converted documents to verify
if news_documents:
    print("Converted News Documents (first 3):")
    for i, doc in enumerate(news_documents[:3]):
        print(f"--- Document {i+1} ---")
        print(f"Page Content: {doc.page_content[:200]}...") # Print first 200 chars of content
        print(f"Metadata: {doc.metadata}")
else:
    print("No news articles were available to convert.")

Converted News Documents (first 3):
--- Document 1 ---
Page Content: トランプ大統領「ウクライナは全領土を取り戻す立場にあると思う」「ロシアは張り子の虎だ」
1 名前：蚤の市 ★：2025/09/24(水) 06:46:15.67 ID:1q0XLAnT9.net







トランプ大統領「ウクライナは全領土を取り戻す立場にあると思う」「ロシアは張り子の虎だ」https://t.co/iIytIcXDzd#国際— 読売新聞オンライン (@Yomiuri_Onli...
Metadata: {'source': 'Itainews.com', 'publishedAt': '2025-09-24T08:02:13Z', 'url': 'https://itainews.com/archives/2053435.html'}
--- Document 2 ---
Page Content: 米国、中印への高関税をG7財務相会合で要請　ロシアへの圧力強化
主要7カ国（G7）が12日に開いたオンライン形式の財務相会合で、米国はロシア産原油の購入を続ける中国やインドに高関税を課すことを提案した。ロシアとの距離が近い中印両国への圧力を、G7を通じて強めたい…...
Metadata: {'source': 'Asahi.com', 'publishedAt': '2025-09-13T05:08:38Z', 'url': 'https://www.asahi.com/articles/AST9F1JH0T9FUHBI00HM.html'}
--- Document 3 ---
Page Content: ロシア無人機、ルーマニアにも領空侵犯か　ウクライナ「偶然でない」
ルーマニア国防省は13日、同国の領空を無人機（ドローン）1機が侵犯したと発表した。国境を接するウクライナにロシアが攻撃中だったといい、ルーマニア外相は侵犯したのはロシアの無人機だとしている。数日前に…...
Metadata: {'source': 'Asahi.com', 'publishedAt': '2025-09-14T11:30:00Z', 'url': 'https://www.asahi.com/articles/AST9G3G

## ドキュメントのチャンク化

### Subtask:
News API から取得したドキュメントを、LLM のコンテキストウィンドウに収まるように小さなチャンクに分割します。


**Reasoning**:
Import the necessary class, define chunk size and overlap, create a text splitter instance, check if news_documents is available, split the documents into chunks, and display the first few chunks.



In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define chunk size and overlap based on LLM context window and desired chunk size
# Referencing the values set in the previous execution, adjusting if necessary.
# The LLM context window is 2048, so smaller chunks are needed.
CHUNK_SIZE = 512
CHUNK_OVERLAP = 128

# Create a RecursiveCharacterTextSplitter instance
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP
)

# Check if news_documents is defined and not empty before splitting
if 'news_documents' in locals() and news_documents:
    # Split the news documents into chunks
    chunked_news_documents = text_splitter.split_documents(news_documents)

    # Display the first few chunked documents to verify
    print("Chunked News Documents (first 3):")
    for i, doc in enumerate(chunked_news_documents[:3]):
        print(f"--- Chunk {i+1} ---")
        print(f"Page Content: {doc.page_content[:200]}...") # Print first 200 chars of content
        print(f"Metadata: {doc.metadata}")
else:
    print("No news documents available to chunk.")
    # Set chunked_news_documents to an empty list if no documents are available
    chunked_news_documents = []


Chunked News Documents (first 3):
--- Chunk 1 ---
Page Content: トランプ大統領「ウクライナは全領土を取り戻す立場にあると思う」「ロシアは張り子の虎だ」
1 名前：蚤の市 ★：2025/09/24(水) 06:46:15.67 ID:1q0XLAnT9.net







トランプ大統領「ウクライナは全領土を取り戻す立場にあると思う」「ロシアは張り子の虎だ」https://t.co/iIytIcXDzd#国際— 読売新聞オンライン (@Yomiuri_Onli...
Metadata: {'source': 'Itainews.com', 'publishedAt': '2025-09-24T08:02:13Z', 'url': 'https://itainews.com/archives/2053435.html'}
--- Chunk 2 ---
Page Content: 米国、中印への高関税をG7財務相会合で要請　ロシアへの圧力強化
主要7カ国（G7）が12日に開いたオンライン形式の財務相会合で、米国はロシア産原油の購入を続ける中国やインドに高関税を課すことを提案した。ロシアとの距離が近い中印両国への圧力を、G7を通じて強めたい…...
Metadata: {'source': 'Asahi.com', 'publishedAt': '2025-09-13T05:08:38Z', 'url': 'https://www.asahi.com/articles/AST9F1JH0T9FUHBI00HM.html'}
--- Chunk 3 ---
Page Content: ロシア無人機、ルーマニアにも領空侵犯か　ウクライナ「偶然でない」
ルーマニア国防省は13日、同国の領空を無人機（ドローン）1機が侵犯したと発表した。国境を接するウクライナにロシアが攻撃中だったといい、ルーマニア外相は侵犯したのはロシアの無人機だとしている。数日前に…...
Metadata: {'source': 'Asahi.com', 'publishedAt': '2025-09-14T11:30:00Z', 'url': 'https://www.asahi.com/articles/AST9G3GVTT9GUHBI00

## ベクトルdb の更新または新規作成

### Subtask:
チャンク化されたニュース記事を既存のベクトルDB に追加するか、新しいベクトルDB を作成して格納します。


**Reasoning**:
The previous command failed because the variable `EMB_MODEL` was not defined. I need to define this variable before instantiating the `HuggingFaceEmbeddings`. I also need to ensure `COLLECTION_NAME` is defined.



In [10]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# Define the embedding model name and collection name
EMB_MODEL = "sentence-transformers/distiluse-base-multilingual-cased-v2"
COLLECTION_NAME = "langchain_news" # Using a different collection name to avoid potential conflicts with previous attempts

# Check if chunked_news_documents is defined and not empty
if 'chunked_news_documents' in locals() and chunked_news_documents:
    # Instantiate the embedding model
    embeddings = HuggingFaceEmbeddings(model_name=EMB_MODEL)

    # Create a new Chroma vector store in memory
    # Using the collection_name variable defined earlier
    vector_store = Chroma.from_documents(
        documents=chunked_news_documents, embedding=embeddings, collection_name=COLLECTION_NAME
    )

    # Print a message indicating success and the number of documents stored
    print(f"Vector store '{COLLECTION_NAME}' created successfully with {len(chunked_news_documents)} documents.")
else:
    # Print a message indicating that the vector store could not be created
    print("No chunked news documents available to create the vector store.")
    # Set vector_store to None if no documents are available
    vector_store = None

Vector store 'langchain_news' created successfully with 51 documents.


## Retriever

### Subtask:
更新または新規作成したベクトルDB を使用する LangChain の Retriever を設定します。


**Reasoning**:
Set up the vector store as a LangChain Retriever if it exists.



In [11]:
# Check if vector_store is defined and not None
if 'vector_store' in locals() and vector_store is not None:
    # Set up the vector DB as a LangChain Retriever
    # Configure the number of chunks to extract using search_kwargs
    retriever = vector_store.as_retriever(search_kwargs={"k": 2})

    print("Retriever set up successfully.")
else:
    print("Vector store is not available. Cannot set up retriever.")
    # Set retriever to None if vector_store is not available
    retriever = None

Retriever set up successfully.


## Langchain チェーンの更新

### Subtask:
既存の LangChain チェーンの Retriever を、News API の結果を含むベクトルDB を使用するものに置き換えます。


**Reasoning**:
Import necessary Langchain classes and define the prompt template.



In [12]:
!pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 283.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 245.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 207.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 262.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 193.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 226.4 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl size=4422321 sha256=7b8dd8751301ce8baa065f6f7d58062ad64dc26ab4fb48dd51ff2e51061b3374
  Stored in directory: /tmp/pip-ephem-wheel-cache-531qlsdv/wheels/90/82/ab/8784ee3fb99ddb07fd36a679ddbe63122cc07718f6c1eb3be8
Successfully built llama-cpp

**Reasoning**:
Now that `llama-cpp-python` is installed, I can retry defining the LLM and constructing the LangChain chain.



In [13]:
from huggingface_hub import hf_hub_download
from langchain_community.llms import LlamaCpp
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Define LLM parameters (referencing values from the original notebook)
CONTEXT_SIZE = 2048
LLM_REPO_ID = "mmnga/ELYZA-japanese-Llama-2-7b-instruct-gguf"
LLM_FILE = "ELYZA-japanese-Llama-2-7b-instruct-q4_K_S.gguf"

# LLMを生成
# Check if model_path is defined, otherwise download it
if 'model_path' not in locals():
    model_path = hf_hub_download(repo_id=LLM_REPO_ID, filename=LLM_FILE)

llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=128,
    n_ctx=CONTEXT_SIZE,
    f16_kv=True,
    verbose=True,
    seed=0
)

# Llama2プロンプトテンプレート
template = """<s>[INST] <<SYS>>
あなたは優秀で中立的な日本人の政治学者です。前提条件の情報だけで回答してください。
<</SYS>>

前提条件：{context}

質問：{question} [/INST]"""

# Create a ChatPromptTemplate instance from the template string
prompt = ChatPromptTemplate.from_template(template)

# Create a StrOutputParser instance
output_parser = StrOutputParser()

# Define the setup_and_retrieval runnable using RunnableParallel
# Assuming 'retriever' is defined and available from a previous step
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

# Construct the LangChain chain by piping the components
chain = setup_and_retrieval | prompt | llm | output_parser

print("LangChain chain updated to use the new retriever.")

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--mmnga--ELYZA-japanese-Llama-2-7b-instruct-gguf/snapshots/2d708f9c52bde588049a494e95b986f5bedba76f/ELYZA-japanese-Llama-2-7b-instruct-q4_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = ELYZA-japanese-Llama-2-7b-instruct
llama_model_loader: - kv   2:       general.source.hugginface.repository str              = elyza/ELYZA-japanese-Llama-2-7b-instruct
llama_model_loader: - kv   3:                   llama.tensor_data_layout str              = Meta AI original pth
llama_model_loader: - kv   4:                       llama.context_length u32              = 4096
llama_model_loader: - kv   5:                     ll

LangChain chain updated to use the new retriever.


CPU : SSE3 = 1 | SSSE3 = 1 | AVX = 1 | AVX2 = 1 | F16C = 1 | FMA = 1 | BMI2 = 1 | LLAMAFILE = 1 | OPENMP = 1 | REPACK = 1 | 
Model metadata: {'general.file_type': '14', 'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'general.architecture': 'llama', 'llama.context_length': '4096', 'general.name': 'ELYZA-japanese-Llama-2-7b-instruct', 'general.source.hugginface.repository': 'elyza/ELYZA-japanese-Llama-2-7b-instruct', 'llama.embedding_length': '4096', 'llama.tensor_data_layout': 'Meta AI original pth', 'llama.feed_forward_length': '11008', 'llama.attention.layer_norm_rms_epsilon': '0.000001', 'llama.rope.dimension_count': '128', 'tokenizer.ggml.bos_token_id': '1', 'llama.attention.head_count': '32', 'llama.block_count': '32', 'llama.attention.head_count_kv': '32', 'general.quantization_version': '2', 'tokenizer.ggml.model': 'llama'}
Using fallback chat format: llama-2


## 質問応答の実行

### Subtask:
更新されたチェーンを使用して、ユーザーの質問に対する回答を生成します。


**Reasoning**:
Generate the response to the user's question using the updated LangChain chain and display the output as a stream.



In [15]:
# ユーザーが質問を入力 (This was done in a previous step and stored in the 'question' variable)
# question = input("質問を入力してください: ")

# Check if the 'chain' object is defined and not None
if 'chain' in locals() and chain is not None:
    print(f"\n回答を生成中...")
    # Invoke the chain with the user's question and stream the output
    for s in chain.stream(question):
        print(s, end="", flush=True)
    print("\n回答生成完了。")
else:
    print("LangChain chain is not available. Cannot generate response.")



回答を生成中...


Llama.generate: 88 prefix-match hit, remaining 895 prompt tokens to eval


  ロシアとウクライナの情勢について回答いたします。

現在、この2国に関する最新情勢は以下の通りです。

- ロシア: プーチン政権によるウクライナ侵略が続き、同国東部の多くの地域が軍事介入以前よりもさらに深刻な状況となっています。また、ロシアはウクライナ領土の大半を侵略し、現在はクリミアを含むそれらの地域を自国領と認知しています。
- ウクライナ: 内戦状態にあるウクライナでは、プーチン政権のロシア軍による侵略以後も

llama_perf_context_print:        load time =  366672.83 ms
llama_perf_context_print: prompt eval time =  338129.49 ms /   895 tokens (  377.80 ms per token,     2.65 tokens per second)
llama_perf_context_print:        eval time =  190465.50 ms /   255 runs   (  746.92 ms per token,     1.34 tokens per second)
llama_perf_context_print:       total time =  529227.72 ms /  1150 tokens
llama_perf_context_print:    graphs reused =        328



回答生成完了。
